In [2]:
!pip install transformers datasets evaluate rouge-score

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import zipfile
import os

# Dézipper le data
zip_file = "/content/drive/MyDrive/zindiHackthonCompetitionLLM2024/DataRename.zip"
output_dir = "/content/data"

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

# Vérifiez le contenu des répertoires
data_json_dir = os.path.join(output_dir, "data_json_indabax_cameroon")
data_pdfs_dir = os.path.join(output_dir, "data_pdfs_indabax_cameroon")

json_files = [os.path.join(data_json_dir, f) for f in os.listdir(data_json_dir) if f.endswith('.json')]
pdf_files = [os.path.join(data_pdfs_dir, f) for f in os.listdir(data_pdfs_dir) if f.endswith('.pdf')]

print(f"Nombre de fichiers JSON: {len(json_files)}")
print(f"Nombre de fichiers PDF: {len(pdf_files)}")


Nombre de fichiers JSON: 1184
Nombre de fichiers PDF: 11


In [4]:
import json
import pandas as pd

data = []

for file in json_files:
    with open(file, 'r') as f:
        json_data = json.load(f)
        if isinstance(json_data, list):
            for item in json_data:
                if isinstance(item, dict):
                    data.append({
                        'title': item.get('title', ''),
                        'content': item.get('content', ''),
                        'detail': item.get('detail', '')
                    })
                else:
                    print(f"Unexpected item type: {type(item)} in file: {file}")
        else:
            print(f"Unexpected json_data type: {type(json_data)} in file: {file}")

json_df = pd.DataFrame(data)
print(json_df.head())


Unexpected json_data type: <class 'dict'> in file: /content/data/data_json_indabax_cameroon/OHADA-COURCOMMUNEDEJUSTICEETDARBITRAGE-20131114-0702013.json
Unexpected json_data type: <class 'dict'> in file: /content/data/data_json_indabax_cameroon/OHADA-COURCOMMUNEDEJUSTICEETDARBITRAGE-20151223-1842015.json
Unexpected json_data type: <class 'dict'> in file: /content/data/data_json_indabax_cameroon/OHADA-COURCOMMUNEDEJUSTICEETDARBITRAGE-20120607-0542012.json
Unexpected json_data type: <class 'dict'> in file: /content/data/data_json_indabax_cameroon/OHADA-COURCOMMUNEDEJUSTICEETDARBITRAGE-20151126-1542015.json
Unexpected json_data type: <class 'dict'> in file: /content/data/data_json_indabax_cameroon/OHADA-COURCOMMUNEDEJUSTICEETDARBITRAGE-20090908-0112012.json
Unexpected json_data type: <class 'dict'> in file: /content/data/data_json_indabax_cameroon/OHADA-COURCOMMUNEDEJUSTICEETDARBITRAGE-20210218-0192021.json
Unexpected json_data type: <class 'dict'> in file: /content/data/data_json_indabax

In [6]:
!pip install PyMuPDF

import fitz  # PyMuPDF
import pandas as pd
pdf_data = []

for pdf_file in pdf_files:
    with fitz.open(pdf_file) as doc:
        text = ""
        for page in doc:
            text += page.get_text()
        pdf_data.append({'file_name': os.path.basename(pdf_file), 'content': text})

pdf_df = pd.DataFrame(pdf_data)
print(pdf_df.head())


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 44.0 MB/s eta 0:00:00
             file_name                                            content
0     AU_mediation.pdf   \nwww.droit‐afrique.com \nOHADA\n  \n \nActe ...
1     AU_arbitrage.pdf   \nwww.droit‐afrique.com \nOHADA\n  \n \nActe ...
2  AU_arbitrage(1).pdf   \n \nwww.Droit-Afrique.com \n \nOHADA \nActe ...
3       AU_ste_com.pdf   \nwww.droit‐afrique.com \nOHADA\n  \n \nActe ...
4       AU_suretes.pdf  www.Droit-Afrique.com \n \nOHADA \n \nActe uni...


In [7]:
combined_df = pd.concat([json_df, pdf_df.rename(columns={'file_name': 'title'})], ignore_index=True)
print(combined_df.head())


                 title                                            content
0     AU_mediation.pdf   \nwww.droit‐afrique.com \nOHADA\n  \n \nActe ...
1     AU_arbitrage.pdf   \nwww.droit‐afrique.com \nOHADA\n  \n \nActe ...
2  AU_arbitrage(1).pdf   \n \nwww.Droit-Afrique.com \n \nOHADA \nActe ...
3       AU_ste_com.pdf   \nwww.droit‐afrique.com \nOHADA\n  \n \nActe ...
4       AU_suretes.pdf  www.Droit-Afrique.com \n \nOHADA \n \nActe uni...


In [8]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(combined_df, test_size=0.1, random_state=42)
print(f"Taille du jeu d'entraînement: {len(train_df)}")
print(f"Taille du jeu de validation: {len(val_df)}")


Taille du jeu d'entraînement: 9
Taille du jeu de validation: 2


In [9]:
eval_file = "/content/drive/MyDrive/zindiHackthonCompetitionLLM2024/Eval.csv"
eval_df = pd.read_csv(eval_file)
print(eval_df.head())


   ID                                           Question  \
0  Q1  Quelles informations doivent être inscrites da...   
1  Q2  Comment s'effectue la demande d'adhésion à une...   
2  Q3  Quel est le processus pour entériner l'adhésio...   
3  Q4  Comment la qualité de coopérateur est-elle off...   
4  Q5  Quelles sont les conditions pour qu'un coopéra...   

                                             Réponse Article Utilisé  \
0  Le registre des membres doit inclure pour chaq...       Article 9   
1  La demande d'adhésion est adressée par écrit à...      Article 10   
2  L'adhésion peut être fixée à la date de la dem...      Article 10   
3  La qualité de coopérateur est constatée par un...      Article 10   
4  Un coopérateur ne peut se retirer qu'après avo...      Article 11   

  Acte uniforme concerné  
0        AU_cooperatives  
1        AU_cooperatives  
2        AU_cooperatives  
3        AU_cooperatives  
4        AU_cooperatives  


In [10]:
!pip install accelerate ###une fois installer redemarrer notre nevironnement

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [11]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset

model_name = "sshleifer/distilbart-cnn-12-6"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Créez des datasets pour les jeux d'entraînement et de validation
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
eval_dataset = Dataset.from_pandas(eval_df)

def preprocess_function(examples):
    if 'title' in examples:
        inputs = [ex for ex in examples['content']]
        labels = [ex for ex in examples['title']]
    elif 'Question' in examples:
        inputs = [ex for ex in examples['Question']]
        labels = [ex for ex in examples['Article Utilisé']]
    else:
        raise KeyError("No suitable column found in the examples dictionary.")

    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(labels, max_length=128, truncation=True, padding=True)

    model_inputs['labels'] = labels['input_ids']
    return model_inputs


train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

# arguments d'entraînement
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/113 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,5.327300
2,No log,3.493177
3,No log,2.940046


TrainOutput(global_step=3, training_loss=7.562011082967122, metrics={'train_runtime': 309.12, 'train_samples_per_second': 0.087, 'train_steps_per_second': 0.01, 'total_flos': 20896813154304.0, 'train_loss': 7.562011082967122, 'epoch': 3.0})

In [ ]:
##########Division et entrainem t de mon model ent 3 et combianison des model obenu ######


# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
# from datasets import Dataset

# model_name = "sshleifer/distilbart-cnn-12-6"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# # Chargement des données
# train_dataset = Dataset.from_pandas(train_df)
# val_dataset = Dataset.from_pandas(val_df)
# eval_dataset = Dataset.from_pandas(eval_df)

# # Définition de la fonction de prétraitement
# def preprocess_function(examples):
#     if 'title' in examples:
#         inputs = [ex for ex in examples['content']]
#         labels = [ex for ex in examples['title']]
#     elif 'Question' in examples:
#         inputs = [ex for ex in examples['Question']]
#         labels = [ex for ex in examples['Article Utilisé']]
#     else:
#         raise KeyError("No suitable column found in the examples dictionary.")

#     model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding=True)

#     with tokenizer.as_target_tokenizer():
#         labels = tokenizer(labels, max_length=128, truncation=True, padding=True)

#     model_inputs['labels'] = labels['input_ids']
#     return model_inputs

# # Prétraitement des données
# train_dataset = train_dataset.map(preprocess_function, batched=True)
# val_dataset = val_dataset.map(preprocess_function, batched=True)
# eval_dataset = eval_dataset.map(preprocess_function, batched=True)

# # Définition des arguments d'entraînement
# training_args = Seq2SeqTrainingArguments(
#     output_dir="./results",
#     evaluation_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     weight_decay=0.01,
#     save_total_limit=3,
#     num_train_epochs=3,
#     predict_with_generate=True,
# )

# # Fonction pour diviser les données et entraîner
# def train_model(train_data, val_data):
#     trainer = Seq2SeqTrainer(
#         model=model,
#         args=training_args,
#         train_dataset=train_data,
#         eval_dataset=val_data,
#         tokenizer=tokenizer,
#     )
#     trainer.train()

# # Diviser les données et entraîner
# num_parts = 3  # Nombre de parties pour diviser les données
# part_size = len(train_dataset) // num_parts

# for i in range(num_parts):
#     start_index = i * part_size
#     end_index = (i + 1) * part_size if i < num_parts - 1 else len(train_dataset)
#     part_train_data = train_dataset.shard(num_shards=num_parts, index=i)  # Divise les données
#     train_model(part_train_data, val_dataset)

# # Combiner les résultats
# from transformers import WEIGHTS_NAME

# merged_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# for i in range(num_parts):

#     part_model = AutoModelForSeq2SeqLM.from_pretrained(f"./results/checkpoint-{i+1}")
#     merged_model = merged_model.__class__.from_pretrained(
#         merged_model.config,
#         state_dict=merged_model.state_dict(),
#         weights=merged_model.config.init_std,
#     )
#     part_state_dict = part_model.state_dict()
#     for name, param in part_state_dict.items():
#         if "lm_head" in name:  # Ne fusionne que les poids du modèle de génération de texte
#             merged_model.state_dict()[name].copy_(param)

# # Utilisez merged_model pour effectuer des prédictions sur de nouvelles données


In [16]:
import evaluate
import numpy as np

rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    prediction_lens = [len(pred) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return result

trainer.evaluate(eval_dataset=eval_dataset, compute_metrics=compute_metrics)


{'eval_loss': 10.41870403289795,
 'eval_runtime': 46.9084,
 'eval_samples_per_second': 2.409,
 'eval_steps_per_second': 0.171,
 'epoch': 3.0}

In [22]:
test_file = "/content/drive/MyDrive/zindiHackthonCompetitionLLM2024/Test.csv"
test_df = pd.read_csv(test_file)
print(test_df.head())

test_dataset = Dataset.from_pandas(test_df)

   ID                                           Question
0  Q1  Quels types d'entités sont soumises aux dispos...
1  Q2  Les sociétés coopératives exerçant une activit...
2  Q3  Dans quels cas les dispositions de l'Acte unif...
3  Q4  Les sociétés coopératives exerçant des activit...
4  Q5  Qui peut choisir de constituer une société coo...


In [26]:
print(test_df.columns)

Index(['ID', 'Question'], dtype='object')


In [27]:
def preprocess_test_function(examples):
    inputs = [ex for ex in examples['Question']]
    labels = [ex for ex in examples['Question']]  # Utiliser 'Question' comme à la fois l'entrée et le label

    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(labels, max_length=128, truncation=True, padding=True)

    model_inputs['labels'] = labels['input_ids']
    return model_inputs


test_dataset = test_dataset.map(preprocess_test_function, batched=True)

predictions = trainer.predict(test_dataset)
predictions_text = tokenizer.batch_decode(predictions.predictions, skip_special_tokens=True)

# Créez le fichier de soumission
submission = pd.DataFrame({
    'ID': test_df['ID'],
    'Answer': predictions_text
})

submission_file = "/content/drive/MyDrive/zindiHackthonCompetitionLLM2024/FirstSubmission.csv"
submission.to_csv(submission_file, index=False)
print("Fichier de soumission créé avec succès!")


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Fichier de soumission créé avec succès!
